question 1: 

Task: use 17 industry portfolio as regressor, estimate HML and MOM 

1. create HML and MOM using 6 portfolios
2. the returns of HML is factor return of value portoflio; the returns of MOM is factor return of momentum portfolio

*portfolio of each year seems to be constructed based on data at the end of June of the same year, so do the same for this?

For each of HML and MOM: 
1. regress 17 industry portfolio on HML/MOM

2. run one of each type of regression: linear, ridge, and LASSO with all regressor. Pick best 2 out 3 using R2, AIC, and BIC 

3. best 2 x (forward + hybrid) + random forest 

  

Compare selection criteria across models and across (1a), (1b), and (1c) 

  

select three best models for HML and for MOM separately for question 1 

  

question 2: 

Task: use 17 industry portfolio as regressor, Comment on the ease or difficulty of replicating the components as opposed to the entire return 

1. use three best model for HML and compare that with accuracy of SV+BV and SG+BG 

2. use three best model for MOM and compare that with accuracy of SH+BH and SL+BL 

  

question 3: 

Task: combine the three procedures, and test accuracy on HML and MOM vs each of the component 

if still have word count, can test accuracy on long-side and short side and/or do pairs combinations of three procedures 


In [55]:
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
from itertools import combinations
import calendar

In [56]:
mom = pd.read_csv("factor_return_mom.csv")

hml = pd.read_csv("factor_return_value.csv")

#The 6 Portfolios Formed on Size and Momentum (contains SH BH SL BL)
size_momentum = pd.read_csv("6_Portfolios_size_mom.csv")
size_momentum = size_momentum.iloc[:1174]
size_momentum['Unnamed: 0'] = size_momentum['Unnamed: 0'].astype(int)

#The 6 Portfolios Formed on Size and Value (contains SV BV SG BG)
size_value = pd.read_csv("6_Portfolios_size_value.csv")
#small lo seems to be small growth

#The 17 Industry Portfolios (regressors)
seventeen = pd.read_csv("17_Industry_Portfolios_cleaned.csv")

# Display the first few rows of the DataFrame
print(hml.head())
print(mom.head())
print(size_momentum.head())
print(size_value.head())
print(seventeen.head())

   Unnamed: 0   HML
0      192607 -2.43
1      192608  3.82
2      192609  0.13
3      192610  0.70
4      192611 -0.51
   Unnamed: 0    MOM
0      192701   0.36
1      192702  -2.14
2      192703   3.61
3      192704   4.30
4      192705  30.00
   Unnamed: 0  SMALL LoPRIOR  ME1 PRIOR2  SMALL HiPRIOR  BIG LoPRIO  \
0      192701           0.08        3.57           0.40       -0.40   
1      192702           7.01        6.48           5.51        7.27   
2      192703          -3.46       -2.88          -2.06       -3.57   
3      192704          -0.75       -0.48           3.32       -2.72   
4      192705           2.48       11.15           6.83        5.52   

   ME2 PRIOR2  B HiPRIOR  
0        0.26       0.00  
1        3.80       4.47  
2       -0.18       2.26  
3        0.75       1.81  
4        4.96       7.18  
   Unnamed: 0  SMALL LoBM  ME1 BM2  SMALL HiBM  BIG LoBM  ME2 BM2  BIG HiBM
0      192607      1.0874   0.9081     -0.0695    5.7168   1.8971    2.0066
1      192608

In [57]:
#make everything start at 192701
hml = hml.iloc[6:]
size_value = size_value.iloc[6:]
seventeen = seventeen.iloc[6:]

print(hml.head())
print(mom.head())
print(size_momentum.head())
print(size_value.head())
print(seventeen.head())

    Unnamed: 0   HML
6       192701  4.54
7       192702  2.94
8       192703 -2.61
9       192704  0.81
10      192705  4.73
   Unnamed: 0    MOM
0      192701   0.36
1      192702  -2.14
2      192703   3.61
3      192704   4.30
4      192705  30.00
   Unnamed: 0  SMALL LoPRIOR  ME1 PRIOR2  SMALL HiPRIOR  BIG LoPRIO  \
0      192701           0.08        3.57           0.40       -0.40   
1      192702           7.01        6.48           5.51        7.27   
2      192703          -3.46       -2.88          -2.06       -3.57   
3      192704          -0.75       -0.48           3.32       -2.72   
4      192705           2.48       11.15           6.83        5.52   

   ME2 PRIOR2  B HiPRIOR  
0        0.26       0.00  
1        3.80       4.47  
2       -0.18       2.26  
3        0.75       1.81  
4        4.96       7.18  
    Unnamed: 0  SMALL LoBM  ME1 BM2  SMALL HiBM  BIG LoBM  ME2 BM2  BIG HiBM
6       192701     -0.7991  -0.3782      3.9316   -0.2621   0.0321    4.0885
7    

In [60]:
size_momentum['Unnamed: 0'] = pd.to_datetime(size_momentum['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')
size_value['Unnamed: 0'] = pd.to_datetime(size_value['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')
hml['Unnamed: 0'] = pd.to_datetime(hml['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')
mom['Unnamed: 0'] = pd.to_datetime(mom['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')
seventeen['Unnamed: 0'] = pd.to_datetime(seventeen['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')

In [61]:
print(seventeen.columns)
seventeen.columns = seventeen.columns.str.strip() #strip spaces
predictors = seventeen.columns[1:]

Index(['Unnamed: 0', 'Food ', 'Mines', 'Oil  ', 'Clths', 'Durbl', 'Chems',
       'Cnsum', 'Cnstr', 'Steel', 'FabPr', 'Machn', 'Cars ', 'Trans', 'Utils',
       'Rtail', 'Finan', 'Other'],
      dtype='object')


In [62]:
# Set date as the index
seventeen.set_index('Unnamed: 0', inplace=True)

In [65]:
data_grouped = seventeen.groupby(pd.Grouper(freq='5Y'))

(Period('1927', '5Y-DEC'),              Food  Mines    Oil  Clths  Durbl  Chems  Cnsum  Cnstr  Steel  \
Unnamed: 0                                                                  
1927-01     -0.82   1.00   1.71  -1.18   4.66   0.40   0.34  -2.78  -0.39   
1927-02      4.56   5.38   0.98   4.15   2.29   9.77   1.60   4.57   3.67   
1927-03      1.81   4.86  -7.00  -0.93  -2.48   6.00   4.86   3.91   1.48   
1927-04      2.73   2.14  -5.81   4.42   2.31   3.09   3.76  -0.71  -0.34   
1927-05      6.21   1.49   4.97   6.00   8.34   4.00  10.85   5.90   3.22   
1927-06     -2.03  -2.54  -3.13   1.49  -3.52  -1.84  -2.08  -4.53  -3.42   
1927-07      5.03   6.06   2.90   5.47   5.93  17.87   4.87   6.74  10.33   
1927-08      2.23   1.54  -0.28   1.22   0.75   2.14   3.47  -0.73   3.97   
1927-09      6.05   4.70   3.43   3.77   2.85   5.88   4.50   5.86   3.77   
1927-10     -2.43  -4.00  -1.85   5.89  -5.34  -7.53   0.68  -4.85  -7.82   
1927-11      7.22   7.95   3.81   3.73   0.31   8

/tmp/ipykernel_1962/980067961.py:1: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  data_grouped = seventeen.groupby(pd.Grouper(freq='5Y'))
/tmp/ipykernel_1962/980067961.py:1: FutureWarning: Resampling with a PeriodIndex is deprecated. Cast index to DatetimeIndex before resampling instead.
  data_grouped = seventeen.groupby(pd.Grouper(freq='5Y'))


In [ ]:
#analysis 1a

linear = LinearRegression()

### forward stepwise selection (use R^2 as criteria for forward stepwise selection)

###### one variable



### backward stepwise selection

### hybrid stepwise selection

#ridge

### forward stepwise selection

### backward stepwise selection

### hybrid stepwise selection

#lasso

### forward stepwise selection

### backward stepwise selection

### hybrid stepwise selection

#random forest